In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import yfinance as yf

In [5]:
ticker_symbol = "RELIANCE.NS"  # Ticker symbol for Reliance Industries
stock_data = yf.download(ticker_symbol, start="2010-01-01", end="2023-06-22")

[*********************100%***********************]  1 of 1 completed


In [6]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,540.427307,540.427307,506.127411,532.700500,482.812469,35372156
2010-01-05,569.551208,569.551208,527.697937,530.323059,480.657684,9872785
2010-01-06,534.879883,542.111328,530.298279,538.891846,488.424011,10933743
2010-01-07,538.891846,552.265076,533.938782,547.832092,496.526886,12090336
2010-01-08,548.797974,551.670715,542.854309,546.395691,495.225098,6973331
...,...,...,...,...,...,...
2023-06-15,2551.800049,2562.550049,2535.500000,2552.550049,2552.550049,5451926
2023-06-16,2560.199951,2582.399902,2560.199951,2577.399902,2577.399902,11155180
2023-06-19,2584.000000,2584.000000,2542.649902,2551.800049,2551.800049,3137532
